In [3]:
import spotipy
import json
import webbrowser
import urllib.request
import spotipy.util as util

#imoort google map
import gmplot
import os
import requests
import googlemaps

from datetime import datetime, timedelta

In [4]:
# get google map key
with open("googlemap_key.txt", "r") as file:
    googlemap_key = file.read()

In [5]:
map_client = googlemaps.Client(googlemap_key)

In [6]:
#google api url to get location for suntime api
#get another city's location for suntime2
ip_address = requests.get('https://api.ipify.org').text
hometown_address = 'Minto,Canada'

In [5]:
#get home town address information
response_hometown_map = map_client.geocode(hometown_address)
print(response_hometown_map)

[{'address_components': [{'long_name': 'Minto', 'short_name': 'Minto', 'types': ['administrative_area_level_3', 'political']}, {'long_name': 'Wellington County', 'short_name': 'Wellington County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Ontario', 'short_name': 'ON', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Canada', 'short_name': 'CA', 'types': ['country', 'political']}, {'long_name': 'N0G', 'short_name': 'N0G', 'types': ['postal_code', 'postal_code_prefix']}], 'formatted_address': 'Minto, ON N0G, Canada', 'geometry': {'bounds': {'northeast': {'lat': 44.004616, 'lng': -80.7418022}, 'southwest': {'lat': 43.8222499, 'lng': -80.99170989999999}}, 'location': {'lat': 43.9134019, 'lng': -80.8717169}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 44.004616, 'lng': -80.7418022}, 'southwest': {'lat': 43.8222499, 'lng': -80.99170989999999}}}, 'place_id': 'ChIJHQ4_rPt8KYgRMUr5wyNNPeA', 'types': ['administrative_area_l

In [6]:
hometown_lattitude = response_hometown_map[0]['geometry']['location']['lat']
hometown_longtitude = response_hometown_map[0]['geometry']['location']['lng']
print(hometown_lattitude)
print(hometown_longtitude)

43.9134019
-80.8717169


In [7]:
url_map = f"https://ipinfo.io/{ip_address}/json"

In [8]:
request_map = urllib.request.Request(url_map)
response_map = urllib.request.urlopen(request_map)

In [9]:
type(response_map)

http.client.HTTPResponse

In [10]:
location_json = json.loads(response_map.read())

In [11]:
lattitude, longtitude = location_json['loc'].split(',')
print(lattitude, longtitude)

51.5085 -0.1257


In [12]:
#use google map current location to get suntime api
url = f"https://api.sunrise-sunset.org/json?lat={lattitude}&lng={longtitude}"
print(url)

https://api.sunrise-sunset.org/json?lat=51.5085&lng=-0.1257


In [13]:
#get hometown's suntime information
url_hometown = f"https://api.sunrise-sunset.org/json?lat={hometown_lattitude}&lng={hometown_longtitude}"
print(url_hometown)

https://api.sunrise-sunset.org/json?lat=43.9134019&lng=-80.8717169


In [14]:
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

#hometown
request_hometown = urllib.request.Request(url_hometown)
response_hometown = urllib.request.urlopen(request_hometown)

In [15]:
type(response)
type(response_hometown)

http.client.HTTPResponse

In [16]:
suntime_json = json.loads(response.read())

In [17]:
print(suntime_json)

{'results': {'sunrise': '7:00:17 AM', 'sunset': '4:27:55 PM', 'solar_noon': '11:44:06 AM', 'day_length': '09:27:38', 'civil_twilight_begin': '6:26:29 AM', 'civil_twilight_end': '5:01:43 PM', 'nautical_twilight_begin': '5:46:39 AM', 'nautical_twilight_end': '5:41:33 PM', 'astronomical_twilight_begin': '5:07:47 AM', 'astronomical_twilight_end': '6:20:25 PM'}, 'status': 'OK', 'tzid': 'UTC'}


In [18]:
hometown_suntime_json = json.loads(response_hometown.read())

In [19]:
print(hometown_suntime_json)

{'results': {'sunrise': '12:04:30 PM', 'sunset': '10:09:41 PM', 'solar_noon': '5:07:06 PM', 'day_length': '10:05:11', 'civil_twilight_begin': '11:35:45 AM', 'civil_twilight_end': '10:38:26 PM', 'nautical_twilight_begin': '11:01:28 AM', 'nautical_twilight_end': '11:12:43 PM', 'astronomical_twilight_begin': '10:27:49 AM', 'astronomical_twilight_end': '11:46:22 PM'}, 'status': 'OK', 'tzid': 'UTC'}


In [20]:
sunrise_hometown = hometown_suntime_json['results']['sunrise']
sunset_hometown = hometown_suntime_json['results']['sunset']

# the printed time is in UK time(8hrs difference)
print(sunrise_hometown)
print(sunset_hometown)

12:04:30 PM
10:09:41 PM


In [21]:
sunrise = suntime_json['results']['sunrise']
sunset = suntime_json['results']['sunset']

print(sunrise)
print(sunset)

7:00:17 AM
4:27:55 PM


In [22]:
# suntime difference

#sunrise
format_str = "%I:%M:%S %p"
sunrise_hometown_dt = datetime.strptime(sunrise_hometown, format_str)
sunrise_dt = datetime.strptime(sunrise, format_str)

if sunrise_dt < sunrise_hometown_dt:
    sunrise_dt += timedelta(days=1)

sunrise_time_difference = sunrise_dt - sunrise_hometown_dt
sunrise_difference = sunrise_time_difference.total_seconds() / 3600  # Convert seconds to hours
print(sunrise_difference)

#sunset
format_str = "%I:%M:%S %p"
sunset_hometown_dt = datetime.strptime(sunset_hometown, format_str)
sunset_dt = datetime.strptime(sunset, format_str)

if sunset_dt < sunset_hometown_dt:
    sunset_dt += timedelta(days=1)

sunset_time_difference = sunset_dt - sunset_hometown_dt
sunset_difference = sunset_time_difference.total_seconds() / 3600  # Convert seconds to hours
print(sunset_difference)

18.92972222222222
18.30388888888889


In [23]:
#time difference set the style of music?
#use numbers after the point, to decide the mood
sunrise_mood = sunrise_difference - 18
print(sunrise_mood)

sunset_mood = sunset_difference - 18
print(sunset_mood)

0.929722222222221
0.30388888888888843


In [24]:
# open spotify api
#ignore the key in git
with open('spotify_keys.json', 'r') as spotify_file:
    tokens = json.load(spotify_file)

In [25]:
client_id = tokens['client_id']
client_secret = tokens['client_secret']
redirectURI = tokens['redirect']
username = tokens['username']

In [26]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-top-read user-library-read user-read-recently-played'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [27]:
# style----decided by time difference
# 20 songs from current position
# 20 songs from home town position
# mix the orders later

#link time difference number with valence, speechiness in Spotify api "tracks"

In [28]:
sp = spotipy.Spotify(auth=token)

In [32]:
# get the most popular singer in current location
#top_playlist = sp.category_playlists(category_id='toplists', country= 'GB')
#playlist_tracks = sp.playlist_tracks(top_playlist['playlists']['items'][0]['id'])

#print(playlist_tracks)

#artists = []
#for item in playlist_tracks['items']:
    #track = item['track']
    #for artist in track['artists']:
        #artists.append(artist['name'])

#top_artists = list(set(artists))
#print(top_artists)

# get the most popular music style in current location


In [33]:
playlist_id = '5ABHKGoOzxkaa28ttQV9sE'
track_results = sp.playlist(playlist_id)
print(track_results)

{'collaborative': False, 'description': 'The 100 most streamed songs on Spotify ever, in correct order (last updated November 4th 2024)', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/5ABHKGoOzxkaa28ttQV9sE'}, 'followers': {'href': None, 'total': 464165}, 'href': 'https://api.spotify.com/v1/playlists/5ABHKGoOzxkaa28ttQV9sE?additional_types=track', 'id': '5ABHKGoOzxkaa28ttQV9sE', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72c82a04c37e7ae0d899edb198e', 'width': None}], 'name': 'Top 100 most streamed songs on Spotify *Updated*', 'owner': {'display_name': 'hoolgerssson', 'external_urls': {'spotify': 'https://open.spotify.com/user/jonathan.holgersson'}, 'href': 'https://api.spotify.com/v1/users/jonathan.holgersson', 'id': 'jonathan.holgersson', 'type': 'user', 'uri': 'spotify:user:jonathan.holgersson'}, 'primary_color': None, 'public': True, 'snapshot_id': 'AAA4C4PUhNuSmu5/I2/uu7ZMId1WmOND', 'tracks': {'href': 'https://api.sp

In [34]:
song_data = track_results['tracks']['items']
print(song_data)

[{'added_at': '2024-02-15T21:57:53Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/jonathan.holgersson'}, 'href': 'https://api.spotify.com/v1/users/jonathan.holgersson', 'id': 'jonathan.holgersson', 'type': 'user', 'uri': 'spotify:user:jonathan.holgersson'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': 'https://p.scdn.co/mp3-preview/51c08d92815cce4ac2de94a7335a430b81234624?cid=8608cbc1f628435c976ab627a00a5230', 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR

In [35]:
songs_id = []
for song in song_data:
    songs_id.append(song['track']['id'])   

songs_id

['0VjIjW4GlUZAMYd2vXMi3b',
 '7qiZfU4dY1lWllzX7mPBI3',
 '7qEHsqek33rTcFNT9PFqLf',
 '4Dvkj6JhhA12EX05fT7y2e',
 '3KkXRkHbMCARz0aVfEt68P',
 '5aAx2yezTd8zXrkmtKl66Z',
 '1xznGGDReH1oQq0xzbwXa3',
 '5PjdY0CKGZdEuoNab3yDmX',
 '2QjOHCTQ1Jl3zawyYOpxh6',
 '0CcQNd8CINkwQfe1RDtGV6',
 '6CDzDgIUqeDY5g8ujExx2f',
 '2XU0oxnq2qxCpomAAuJY8K',
 '0tgVpDi06FyKpA1z0VMD4v',
 '7wGoVu4Dady5GV0Sv4UIsx',
 '2NQzj8czPzUw9ufCOvAT8W',
 '0u2P5u6lvoDfwTYjAADbn4',
 '5uCax9HTNlzGybIStD3vDh',
 '7BKLCZ1jbUBVqRi2FVlTVw',
 '6UelLqGlWMcVH1E5c4H7lY',
 '0TK2YIli7K1leLovkQiNik',
 '6RUKPb4LETWmmr3iAEQktW',
 '6EIsrLeTHnX1fNU2Ci36RL',
 '7yq4Qj7cqayVTp3FF9CWbm',
 '3PfIrDoz19wz7qK7tYeu62',
 '2EiGECydkS2M8OCcRHQZhT',
 '3JvKfv6T31zO0ini8iNItO',
 '5edBgVtRD0fvWk140Sl21T',
 '1Bk4mFK1shZUoHfYJMwqWp',
 '1HNkqx9Ahdgi1Ixy2xkKkL',
 '6krwVrqihThMtvID5LlaYj',
 '285pBltuF7vW8TeWk8hdRR',
 '21jGcNKet2qwijlDFuPiPb',
 '2snvV5XQw30jstCYlr1W5o',
 '6T8cJz5lAqGer9GUHGyelE',
 '4u7EnebtmKWzUH433cf5Qv',
 '2Fxmhks0bxGSBdJ92vM42m',
 '34gCuhDGsG4bRPIf9bb02f',
 

In [36]:
song_feature = sp.audio_features(list(songs_id))
print(song_feature)

[{'danceability': 0.513, 'energy': 0.73, 'key': 1, 'loudness': -5.94, 'mode': 1, 'speechiness': 0.0598, 'acousticness': 0.00143, 'instrumentalness': 9.54e-05, 'liveness': 0.0897, 'valence': 0.334, 'tempo': 171.001, 'type': 'audio_features', 'id': '0VjIjW4GlUZAMYd2vXMi3b', 'uri': 'spotify:track:0VjIjW4GlUZAMYd2vXMi3b', 'track_href': 'https://api.spotify.com/v1/tracks/0VjIjW4GlUZAMYd2vXMi3b', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0VjIjW4GlUZAMYd2vXMi3b', 'duration_ms': 200040, 'time_signature': 4}, {'danceability': 0.825, 'energy': 0.652, 'key': 1, 'loudness': -3.183, 'mode': 0, 'speechiness': 0.0802, 'acousticness': 0.581, 'instrumentalness': 0, 'liveness': 0.0931, 'valence': 0.931, 'tempo': 95.977, 'type': 'audio_features', 'id': '7qiZfU4dY1lWllzX7mPBI3', 'uri': 'spotify:track:7qiZfU4dY1lWllzX7mPBI3', 'track_href': 'https://api.spotify.com/v1/tracks/7qiZfU4dY1lWllzX7mPBI3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7qiZfU4dY1lWllzX7mPBI3', 'duratio

In [37]:
song_valence = []

for song in song_feature:
    song_valence.append(song['valence'])   

song_valence

[0.334,
 0.931,
 0.446,
 0.662,
 0.913,
 0.535,
 0.371,
 0.478,
 0.398,
 0.708,
 0.531,
 0.513,
 0.168,
 0.119,
 0.598,
 0.12,
 0.494,
 0.661,
 0.557,
 0.75,
 0.446,
 0.699,
 0.51,
 0.679,
 0.405,
 0.131,
 0.465,
 0.869,
 0.201,
 0.657,
 0.218,
 0.551,
 0.595,
 0.318,
 0.226,
 0.562,
 0.591,
 0.847,
 0.552,
 0.285,
 0.43,
 0.331,
 0.564,
 0.166,
 0.515,
 0.479,
 0.298,
 0.331,
 0.0998,
 0.824,
 0.507,
 0.643,
 0.416,
 0.109,
 0.163,
 0.648,
 0.417,
 0.494,
 0.423,
 0.204,
 0.244,
 0.33,
 0.236,
 0.662,
 0.059,
 0.411,
 0.405,
 0.424,
 0.137,
 0.756,
 0.592,
 0.771,
 0.688,
 0.473,
 0.159,
 0.733,
 0.632,
 0.4,
 0.437,
 0.74,
 0.86,
 0.671,
 0.87,
 0.401,
 0.446,
 0.41,
 0.274,
 0.608,
 0.969,
 0.545,
 0.915,
 0.72,
 0.942,
 0.474,
 0.377,
 0.18,
 0.43,
 0.629,
 0.394,
 0.456]

In [46]:
# find valence in song_feature which = sunrise_mood
sunrise_matching_songs = [song for song in song_feature if round(song.get('valence'),1) == round(abs(sunrise_mood),1)]
sunset_matching_songs = [song for song in song_feature if round(song.get('valence'),1) == round(abs(sunset_mood),1)]
print(sunrise_matching_songs)
print(sunset_matching_songs)

[{'danceability': 0.825, 'energy': 0.652, 'key': 1, 'loudness': -3.183, 'mode': 0, 'speechiness': 0.0802, 'acousticness': 0.581, 'instrumentalness': 0, 'liveness': 0.0931, 'valence': 0.931, 'tempo': 95.977, 'type': 'audio_features', 'id': '7qiZfU4dY1lWllzX7mPBI3', 'uri': 'spotify:track:7qiZfU4dY1lWllzX7mPBI3', 'track_href': 'https://api.spotify.com/v1/tracks/7qiZfU4dY1lWllzX7mPBI3', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7qiZfU4dY1lWllzX7mPBI3', 'duration_ms': 233713, 'time_signature': 4}, {'danceability': 0.76, 'energy': 0.478, 'key': 2, 'loudness': -5.574, 'mode': 1, 'speechiness': 0.0467, 'acousticness': 0.552, 'instrumentalness': 0, 'liveness': 0.0703, 'valence': 0.913, 'tempo': 89.908, 'type': 'audio_features', 'id': '3KkXRkHbMCARz0aVfEt68P', 'uri': 'spotify:track:3KkXRkHbMCARz0aVfEt68P', 'track_href': 'https://api.spotify.com/v1/tracks/3KkXRkHbMCARz0aVfEt68P', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3KkXRkHbMCARz0aVfEt68P', 'duration_ms': 15

In [47]:
#print(my_sunrise_playlist)

In [48]:
#sunset_song_uris = []
#for song in sunset_matching_songs:
    #sunset_song_uris.append(song['uri'])
#print(sunset_song_uris)

In [49]:
#my_sunset_playlist = sp.user_playlist_create(user=username, name=sunset, public=True, description="songs for the sunrise")

In [50]:
#print(my_sunset_playlist)

In [51]:
# hometown popular song list
#hometown_top_playlist = sp.category_playlists(category_id='toplists', country= 'CN')
#hometown_playlist_tracks = sp.playlist_tracks(top_playlist['playlists']['items'][0]['id'])

#print(hometown_playlist_tracks)

In [52]:
playlist_id = '37i9dQZEVXbKj23U1GF4IR'
hometown_playlist_tracks = sp.playlist(playlist_id)
print(hometown_playlist_tracks)

{'collaborative': False, 'description': 'Your daily update of the most played tracks right now - Canada.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZEVXbKj23U1GF4IR'}, 'followers': {'href': None, 'total': 333826}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZEVXbKj23U1GF4IR?additional_types=track', 'id': '37i9dQZEVXbKj23U1GF4IR', 'images': [{'height': None, 'url': 'https://charts-images.scdn.co/assets/locale_en/regional/daily/region_ca_default.jpg', 'width': None}], 'name': 'Top 50 - Canada', 'owner': {'display_name': 'Spotify', 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'}, 'href': 'https://api.spotify.com/v1/users/spotify', 'id': 'spotify', 'type': 'user', 'uri': 'spotify:user:spotify'}, 'primary_color': None, 'public': True, 'snapshot_id': 'Lrz40AAAAAAAAAAAAAAAAAAAAAAAAGVu', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZEVXbKj23U1GF4IR/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_a

In [53]:
hometown_song_data = hometown_playlist_tracks['tracks']['items']
print(hometown_song_data)

[{'added_at': '2024-11-05T12:26:47Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'}, 'href': 'https://api.spotify.com/v1/users/', 'id': '', 'type': 'user', 'uri': 'spotify:user:'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': 'https://p.scdn.co/mp3-preview/aed073ed2859447c5b275cd7bace28302330b629?cid=8608cbc1f628435c976ab627a00a5230', 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', '

In [54]:
hometown_id = []
for song in hometown_song_data:
    hometown_id.append(song['track']['id'])   

hometown_id

['7ne4VBA60CxGM75vw0EYad',
 '5vNRhkKd0yEAg8suGBpjeY',
 '2plbrEY59IikOBgBGLjaoe',
 '2262bWmqomIaJXwCRHr13j',
 '1Es7AUAhQvapIcoh3qMKDL',
 '6dOtVTDdiauQNBQEDOtlAB',
 '5G2f63n7IPVPPjfNIGih7Q',
 '5fZJQrFKWQLb7FpJXZ1g7K',
 '1QoyuMHNBe7lg3YW4Qtll4',
 '0WbMK4wrZ1wFSty9F7FCgu',
 '7hR22TOX3RorxJPcsz5Wbo',
 '3xkHsmpQCBMytMJNiDf3Ii',
 '5IZXB5IKAD2qlvTPJYDCFB',
 '2HRqTpkrJO5ggZyyK6NPWz',
 '51rfRCiUSvxXlCSCfIztBy',
 '0VaeksJaXy5R1nvcTMh3Xk',
 '3tFed7YsjGnIfxeLEQwx3R',
 '6jbYpRPTEFl1HFKHk1IC0m',
 '3QaPy1KgI7nu9FJEQUgn6h',
 '2tHwzyyOLoWSFqYNjeVMzj',
 '0mflMxspEfB0VbI1kyLiAv',
 '17phhZDn6oGtzMe56NuWvj',
 '4KULAymBBJcPRpk1yO4dOG',
 '5RePVWy39tLpHH0WwXgBsK',
 '2uqYupMHANxnwgeiXTZXzd',
 '1UHS8Rf6h5Ar3CDWRd3wjF',
 '4ZJ4vzLQekI0WntDbanNC7',
 '1YdtGoSYDvVQKoUqSDXX41',
 '3WMj8moIAXJhHsyLaqIIHI',
 '7tI8dRuH2Yc6RuoTjxo4dU',
 '7KA4W4McWYRpgf0fWsJZWB',
 '19XpFsce28aByvCC4g89tJ',
 '1XBYiRV30ykHw5f4wm6qEn',
 '2aYHxnMF2umAavtgBvmkY1',
 '0cQVqPuHQP4KEwc7ZUQmj6',
 '3Vr3zh0r7ALn8VLqCiRR10',
 '0xGQTpN4ESETtl1Sp6rzd2',
 

In [55]:
hometown_feature = sp.audio_features(list(hometown_id))
print(hometown_feature)

[{'danceability': 0.554, 'energy': 0.808, 'key': 1, 'loudness': -4.169, 'mode': 1, 'speechiness': 0.0368, 'acousticness': 0.214, 'instrumentalness': 0, 'liveness': 0.159, 'valence': 0.372, 'tempo': 108.548, 'type': 'audio_features', 'id': '7ne4VBA60CxGM75vw0EYad', 'uri': 'spotify:track:7ne4VBA60CxGM75vw0EYad', 'track_href': 'https://api.spotify.com/v1/tracks/7ne4VBA60CxGM75vw0EYad', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7ne4VBA60CxGM75vw0EYad', 'duration_ms': 166300, 'time_signature': 4}, {'danceability': 0.777, 'energy': 0.783, 'key': 0, 'loudness': -4.477, 'mode': 0, 'speechiness': 0.26, 'acousticness': 0.0283, 'instrumentalness': 0, 'liveness': 0.355, 'valence': 0.939, 'tempo': 149.027, 'type': 'audio_features', 'id': '5vNRhkKd0yEAg8suGBpjeY', 'uri': 'spotify:track:5vNRhkKd0yEAg8suGBpjeY', 'track_href': 'https://api.spotify.com/v1/tracks/5vNRhkKd0yEAg8suGBpjeY', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5vNRhkKd0yEAg8suGBpjeY', 'duration_ms': 16

In [56]:
# find valence in song_feature which = sunrise_mood
hometown_sunrise_matching_songs = [song for song in hometown_feature if round(song.get('valence'),1) == round(abs(sunrise_mood),1)]
hometown_sunset_matching_songs = [song for song in hometown_feature if round(song.get('valence'),1) == round(abs(sunset_mood),1)]
print(hometown_sunrise_matching_songs)
print(hometown_sunset_matching_songs)

[{'danceability': 0.777, 'energy': 0.783, 'key': 0, 'loudness': -4.477, 'mode': 0, 'speechiness': 0.26, 'acousticness': 0.0283, 'instrumentalness': 0, 'liveness': 0.355, 'valence': 0.939, 'tempo': 149.027, 'type': 'audio_features', 'id': '5vNRhkKd0yEAg8suGBpjeY', 'uri': 'spotify:track:5vNRhkKd0yEAg8suGBpjeY', 'track_href': 'https://api.spotify.com/v1/tracks/5vNRhkKd0yEAg8suGBpjeY', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5vNRhkKd0yEAg8suGBpjeY', 'duration_ms': 169917, 'time_signature': 4}, {'danceability': 0.852, 'energy': 0.697, 'key': 1, 'loudness': -5.52, 'mode': 0, 'speechiness': 0.0439, 'acousticness': 0.0973, 'instrumentalness': 0.00037, 'liveness': 0.0678, 'valence': 0.919, 'tempo': 138.029, 'type': 'audio_features', 'id': '5AJ9hqTS2wcFQCELCFRO7A', 'uri': 'spotify:track:5AJ9hqTS2wcFQCELCFRO7A', 'track_href': 'https://api.spotify.com/v1/tracks/5AJ9hqTS2wcFQCELCFRO7A', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5AJ9hqTS2wcFQCELCFRO7A', 'duration_

In [57]:
sunrise_song_uris = []
for song in sunrise_matching_songs:
    sunrise_song_uris.append(song['uri'])
    
for song in hometown_sunrise_matching_songs:
    sunrise_song_uris.append(song['uri'])
    
print(sunrise_song_uris)

['spotify:track:7qiZfU4dY1lWllzX7mPBI3', 'spotify:track:3KkXRkHbMCARz0aVfEt68P', 'spotify:track:1Bk4mFK1shZUoHfYJMwqWp', 'spotify:track:0KKkJNfGyhkQ5aFogxQAPU', 'spotify:track:5g7sDjBhZ4I3gcFIpkrLuI', 'spotify:track:463CkQjx2Zk1yXoBuierM9', 'spotify:track:6zSpb8dQRaw0M1dK8PBwQz', 'spotify:track:5vNRhkKd0yEAg8suGBpjeY', 'spotify:track:5AJ9hqTS2wcFQCELCFRO7A']


In [58]:
my_sunrise_playlist = sp.user_playlist_create(user=username, name=sunrise, public=True, description="songs for the sunrise")

In [59]:
print(my_sunrise_playlist)

{'collaborative': False, 'description': 'songs for the sunrise', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/7thShw3gpQTp30tdNLRb4l'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/7thShw3gpQTp30tdNLRb4l', 'id': '7thShw3gpQTp30tdNLRb4l', 'images': [], 'primary_color': None, 'name': '7:00:17 AM', 'type': 'playlist', 'uri': 'spotify:playlist:7thShw3gpQTp30tdNLRb4l', 'owner': {'href': 'https://api.spotify.com/v1/users/31nmjw7jeefxnmo2xlswxclw4v6y', 'id': '31nmjw7jeefxnmo2xlswxclw4v6y', 'type': 'user', 'uri': 'spotify:user:31nmjw7jeefxnmo2xlswxclw4v6y', 'display_name': None, 'external_urls': {'spotify': 'https://open.spotify.com/user/31nmjw7jeefxnmo2xlswxclw4v6y'}}, 'public': True, 'snapshot_id': 'AAAARJBLVHh7XXQZUCiqc94R1Nk62OW5', 'tracks': {'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'href': 'https://api.spotify.com/v1/playlists/7thShw3gpQTp30tdNLRb4l/tracks', 'total': 0, 'items': []}}


In [60]:
sunset_song_uris = []
for song in sunset_matching_songs:
    sunset_song_uris.append(song['uri'])
    
for song in hometown_sunset_matching_songs:
    sunset_song_uris.append(song['uri'])
    
print(sunset_song_uris)

['spotify:track:0VjIjW4GlUZAMYd2vXMi3b', 'spotify:track:6T8cJz5lAqGer9GUHGyelE', 'spotify:track:3AJwUDP919kvQ9QcozQPxg', 'spotify:track:2VxeLyX666F8uXCJ0dZF8B', 'spotify:track:0tKcYR2II1VCQWT79i5NrW', 'spotify:track:3U4isOIWM3VvDubwSI3y7a', 'spotify:track:6ocbgoVGwYJhOv1GgI9NsF', 'spotify:track:4WjH9Bzt3kx7z8kl0awxh4', 'spotify:track:2262bWmqomIaJXwCRHr13j', 'spotify:track:51rfRCiUSvxXlCSCfIztBy', 'spotify:track:4ZJ4vzLQekI0WntDbanNC7', 'spotify:track:3Vr3zh0r7ALn8VLqCiRR10', 'spotify:track:2c7z5oRcPBbqRaBY2mLWcf', 'spotify:track:6GG4yyk3UATdBfTHVgI8PB']


In [61]:
my_sunset_playlist = sp.user_playlist_create(user=username, name=sunset, public=True, description="songs for the sunset")

In [62]:
print(my_sunset_playlist)

{'collaborative': False, 'description': 'songs for the sunset', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/5J5kFDp0ariYocq876XuXi'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/5J5kFDp0ariYocq876XuXi', 'id': '5J5kFDp0ariYocq876XuXi', 'images': [], 'primary_color': None, 'name': '4:27:55 PM', 'type': 'playlist', 'uri': 'spotify:playlist:5J5kFDp0ariYocq876XuXi', 'owner': {'href': 'https://api.spotify.com/v1/users/31nmjw7jeefxnmo2xlswxclw4v6y', 'id': '31nmjw7jeefxnmo2xlswxclw4v6y', 'type': 'user', 'uri': 'spotify:user:31nmjw7jeefxnmo2xlswxclw4v6y', 'display_name': None, 'external_urls': {'spotify': 'https://open.spotify.com/user/31nmjw7jeefxnmo2xlswxclw4v6y'}}, 'public': True, 'snapshot_id': 'AAAARW/4+ylYWHb75EyNwK6rM4H3Y/U4', 'tracks': {'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'href': 'https://api.spotify.com/v1/playlists/5J5kFDp0ariYocq876XuXi/tracks', 'total': 0, 'items': []}}


In [63]:
sp.user_playlist_add_tracks(username, my_sunrise_playlist['id'], sunrise_song_uris)

{'snapshot_id': 'AAAAAllOc+zxqMt/RQ4HltZ6qDdoCp7z'}

In [64]:
sp.user_playlist_add_tracks(username, my_sunset_playlist['id'], sunset_song_uris)

{'snapshot_id': 'AAAAAl+drptfmlC3WxGcYY8/UO3sHXdn'}

In [65]:
webbrowser.open(my_sunrise_playlist['external_urls']['spotify'])
webbrowser.open(my_sunset_playlist['external_urls']['spotify'])

True